# 25 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier

2023-11-29 10:16:10.666121: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 10:16:10.702142: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 10:16:10.702188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 10:16:10.703690: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 10:16:10.712088: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 10:16:10.713072: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [2]:
no_of_patients=25

In [3]:
train=sorted(glob('files/*.edf'))
train=train[:no_of_patients]
train

['files/S001R03.edf',
 'files/S002R03.edf',
 'files/S003R03.edf',
 'files/S004R03.edf',
 'files/S005R03.edf',
 'files/S006R03.edf',
 'files/S007R03.edf',
 'files/S008R03.edf',
 'files/S009R03.edf',
 'files/S010R03.edf',
 'files/S011R03.edf',
 'files/S012R03.edf',
 'files/S013R03.edf',
 'files/S014R03.edf',
 'files/S015R03.edf',
 'files/S016R03.edf',
 'files/S017R03.edf',
 'files/S018R03.edf',
 'files/S019R03.edf',
 'files/S020R03.edf',
 'files/S021R03.edf',
 'files/S022R03.edf',
 'files/S023R03.edf',
 'files/S024R03.edf',
 'files/S025R03.edf']

In [4]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [5]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [6]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [7]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [8]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(300000, 37475, 300000, 37475)

In [9]:
print(xtest.iloc[:,-1].values)

[-5.7e-05 -4.5e-05 -2.9e-05 ... -1.1e-05 -1.1e-05 -1.2e-05]


In [10]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,-0.000005,-0.000025,-0.000043,0.000008,-0.000020,-0.000024,-0.000028,-0.000019,-0.000024,-0.000022,...,0.000003,-0.000007,-0.000014,-0.000004,-0.000009,-0.000003,0.000001,0.000001,-0.000004,0.000005
299996,-0.000013,-0.000029,-0.000046,0.000006,-0.000021,-0.000020,-0.000010,-0.000026,-0.000027,-0.000025,...,0.000001,-0.000016,-0.000020,-0.000005,-0.000011,-0.000008,-0.000002,-0.000004,-0.000010,-0.000001
299997,-0.000021,-0.000029,-0.000045,0.000008,-0.000017,-0.000014,-0.000005,-0.000028,-0.000028,-0.000026,...,-0.000005,-0.000008,-0.000015,-0.000004,-0.000011,-0.000008,0.000000,-0.000003,-0.000012,0.000000
299998,-0.000003,-0.000022,-0.000039,0.000013,-0.000012,-0.000010,-0.000012,-0.000020,-0.000021,-0.000021,...,-0.000005,-0.000005,-0.000011,0.000001,-0.000005,0.000001,0.000004,-0.000001,-0.000008,0.000002


In [11]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [12]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [13]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_11878/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_11878/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_11878/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,-0.000005,-0.000025,-0.000043,0.000008,-0.000020,-0.000024,-0.000028,-0.000019,-0.000024,-0.000022,...,0.000003,-0.000007,-0.000014,-0.000004,-0.000009,-0.000003,0.000001,0.000001,-0.000004,0.000005
299996,-0.000013,-0.000029,-0.000046,0.000006,-0.000021,-0.000020,-0.000010,-0.000026,-0.000027,-0.000025,...,0.000001,-0.000016,-0.000020,-0.000005,-0.000011,-0.000008,-0.000002,-0.000004,-0.000010,-0.000001
299997,-0.000021,-0.000029,-0.000045,0.000008,-0.000017,-0.000014,-0.000005,-0.000028,-0.000028,-0.000026,...,-0.000005,-0.000008,-0.000015,-0.000004,-0.000011,-0.000008,0.000000,-0.000003,-0.000012,0.000000
299998,-0.000003,-0.000022,-0.000039,0.000013,-0.000012,-0.000010,-0.000012,-0.000020,-0.000021,-0.000021,...,-0.000005,-0.000005,-0.000011,0.000001,-0.000005,0.000001,0.000004,-0.000001,-0.000008,0.000002


In [14]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [16]:
model=XGBClassifier(n_estimators=500)
model.fit(x8,y8,early_stopping_rounds=10, eval_set=[(xv8, yv8)])
y_pred=model.predict(xt8)
print(classification_report(yt8,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.96110
[1]	validation_0-mlogloss:2.85396
[2]	validation_0-mlogloss:2.77735
[3]	validation_0-mlogloss:2.71245
[4]	validation_0-mlogloss:2.66136
[5]	validation_0-mlogloss:2.61738
[6]	validation_0-mlogloss:2.57786
[7]	validation_0-mlogloss:2.54553
[8]	validation_0-mlogloss:2.51563
[9]	validation_0-mlogloss:2.48513
[10]	validation_0-mlogloss:2.46291
[11]	validation_0-mlogloss:2.44237
[12]	validation_0-mlogloss:2.42190
[13]	validation_0-mlogloss:2.40400
[14]	validation_0-mlogloss:2.38842
[15]	validation_0-mlogloss:2.37250
[16]	validation_0-mlogloss:2.35562
[17]	validation_0-mlogloss:2.34300
[18]	validation_0-mlogloss:2.33208
[19]	validation_0-mlogloss:2.31765
[20]	validation_0-mlogloss:2.30447
[21]	validation_0-mlogloss:2.29316
[22]	validation_0-mlogloss:2.28260
[23]	validation_0-mlogloss:2.27078
[24]	validation_0-mlogloss:2.25778
[25]	validation_0-mlogloss:2.24768
[26]	validation_0-mlogloss:2.23702
[27]	validation_0-mlogloss:2.22651
[28]	validation_0-mlogloss:2.2

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
9375/9375 [==============================] - 14s 1ms/step - loss: 1.8391 - val_loss: 1.9462
Epoch 2/10
9375/9375 [==============================] - 12s 1ms/step - loss: 1.4581 - val_loss: 1.9352
Epoch 3/10
9375/9375 [==============================] - 13s 1ms/step - loss: 1.3045 - val_loss: 1.7063
Epoch 4/10
9375/9375 [==============================] - 12s 1ms/step - loss: 1.1785 - val_loss: 1.7581
Epoch 5/10
9375/9375 [==============================] - 13s 1ms/step - loss: 1.1359 - val_loss: 1.8647
Epoch 6/10
9375/9375 [==============================] - 12s 1ms/step - loss: 1.1074 - val_loss: 1.8822
Epoch 7/10
9375/9375 [==============================] - 12s 1ms/step - loss: 1.0904 - val_loss: 1.9826
Epoch 8/10
9375/9375 [==============================] - 12s 1ms/step - loss: 1.0737 - val_loss: 1.8391
Epoch 9/10
9375/9375 [==============================] - 12s 1ms/step - loss: 1.0607 - val_loss: 1.9776
Epoch 10/10
9375/9375 [==============================] - 12s 1ms/step - l

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

 186/1172 [===>..........................] - ETA: 0s

1172/1172 [==============================] - 1s 801us/step
              precision    recall  f1-score   support

           0       0.32      0.25      0.28      1499
           1       0.27      0.41      0.32      1499
           2       0.38      0.50      0.43      1499
           3       0.61      0.82      0.70      1499
           4       0.75      0.68      0.71      1499
           5       0.39      0.53      0.45      1499
           6       0.45      0.17      0.25      1499
           7       0.53      0.35      0.42      1499
           8       0.45      0.78      0.57      1499
           9       0.67      0.65      0.66      1499
          10       0.28      0.24      0.26      1499
          11       0.22      0.32      0.26      1499
          12       0.47      0.29      0.36      1499
          13       0.22      0.28      0.25      1499
          14       0.33      0.22      0.27      1499
          15       0.44      0.22      0.29      1499
          16       0.1

## 0-16

In [17]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [18]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_11878/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_11878/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_11878/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,-0.000005,-0.000025,-0.000043,0.000008,-0.000020,-0.000024,-0.000028,-0.000019,-0.000024,-0.000022,...,0.000003,-0.000007,-0.000014,-0.000004,-0.000009,-0.000003,0.000001,0.000001,-0.000004,0.000005
299996,-0.000013,-0.000029,-0.000046,0.000006,-0.000021,-0.000020,-0.000010,-0.000026,-0.000027,-0.000025,...,0.000001,-0.000016,-0.000020,-0.000005,-0.000011,-0.000008,-0.000002,-0.000004,-0.000010,-0.000001
299997,-0.000021,-0.000029,-0.000045,0.000008,-0.000017,-0.000014,-0.000005,-0.000028,-0.000028,-0.000026,...,-0.000005,-0.000008,-0.000015,-0.000004,-0.000011,-0.000008,0.000000,-0.000003,-0.000012,0.000000
299998,-0.000003,-0.000022,-0.000039,0.000013,-0.000012,-0.000010,-0.000012,-0.000020,-0.000021,-0.000021,...,-0.000005,-0.000005,-0.000011,0.000001,-0.000005,0.000001,0.000004,-0.000001,-0.000008,0.000002


In [19]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [20]:
model2=XGBClassifier(n_estimators=500)
model2.fit(x16,y16,early_stopping_rounds=10, eval_set=[(xv16, yv16)])
y_pred=model2.predict(xt16)
print(classification_report(yt16,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.91459
[1]	validation_0-mlogloss:2.76978
[2]	validation_0-mlogloss:2.67245
[3]	validation_0-mlogloss:2.59598
[4]	validation_0-mlogloss:2.53259
[5]	validation_0-mlogloss:2.47844
[6]	validation_0-mlogloss:2.43092
[7]	validation_0-mlogloss:2.38483
[8]	validation_0-mlogloss:2.34447
[9]	validation_0-mlogloss:2.30765
[10]	validation_0-mlogloss:2.28139
[11]	validation_0-mlogloss:2.25515
[12]	validation_0-mlogloss:2.22975
[13]	validation_0-mlogloss:2.20087
[14]	validation_0-mlogloss:2.17669
[15]	validation_0-mlogloss:2.15359
[16]	validation_0-mlogloss:2.13052
[17]	validation_0-mlogloss:2.11277
[18]	validation_0-mlogloss:2.09711
[19]	validation_0-mlogloss:2.08169
[20]	validation_0-mlogloss:2.06526
[21]	validation_0-mlogloss:2.04576
[22]	validation_0-mlogloss:2.03038
[23]	validation_0-mlogloss:2.01305
[24]	validation_0-mlogloss:2.00159
[25]	validation_0-mlogloss:1.98977
[26]	validation_0-mlogloss:1.97729
[27]	validation_0-mlogloss:1.96178
[28]	validation_0-mlogloss:1.9

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10


9375/9375 [==============================] - 13s 1ms/step - loss: 1.2853 - val_loss: 1.8553
Epoch 2/10
9375/9375 [==============================] - 12s 1ms/step - loss: 0.7151 - val_loss: 1.9334
Epoch 3/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.5800 - val_loss: 2.0008
Epoch 4/10
9375/9375 [==============================] - 12s 1ms/step - loss: 0.5135 - val_loss: 2.3795
Epoch 5/10
9375/9375 [==============================] - 12s 1ms/step - loss: 0.4749 - val_loss: 2.1545
Epoch 6/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.4481 - val_loss: 2.2653
Epoch 7/10
9375/9375 [==============================] - 12s 1ms/step - loss: 0.4290 - val_loss: 2.2852
Epoch 8/10
9375/9375 [==============================] - 12s 1ms/step - loss: 0.4122 - val_loss: 2.2804
Epoch 9/10
9375/9375 [==============================] - 12s 1ms/step - loss: 0.3997 - val_loss: 2.3937
Epoch 10/10
9375/9375 [==============================] - 12s 1ms/step - loss: 0.3886

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

1172/1172 [==============================] - 1s 797us/step


              precision    recall  f1-score   support

           0       0.52      0.48      0.50      1499
           1       0.43      0.63      0.51      1499
           2       0.40      0.42      0.41      1499
           3       0.77      0.64      0.70      1499
           4       0.79      0.88      0.84      1499
           5       0.63      0.66      0.65      1499
           6       0.94      0.52      0.67      1499
           7       0.87      0.84      0.85      1499
           8       0.56      0.86      0.68      1499
           9       0.65      0.77      0.71      1499
          10       0.74      0.25      0.37      1499
          11       0.66      0.39      0.49      1499
          12       0.50      0.54      0.52      1499
          13       0.28      0.55      0.37      1499
          14       0.37      0.48      0.41      1499
          15       0.72      0.09      0.16      1499
          16       0.62      0.55      0.58      1499
          17       0.98    

## 0-32

In [21]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [22]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_11878/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_11878/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_11878/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [23]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [25]:
model3=XGBClassifier(n_estimators=500)
model3.fit(x32,y32,early_stopping_rounds=10, eval_set=[(xv32, yv32)])
y_pred=model3.predict(xt32)
print(classification_report(yt32,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.71878
[1]	validation_0-mlogloss:2.50676
[2]	validation_0-mlogloss:2.36490
[3]	validation_0-mlogloss:2.24633
[4]	validation_0-mlogloss:2.15235
[5]	validation_0-mlogloss:2.07357
[6]	validation_0-mlogloss:1.99772
[7]	validation_0-mlogloss:1.93303
[8]	validation_0-mlogloss:1.87601
[9]	validation_0-mlogloss:1.82581
[10]	validation_0-mlogloss:1.78118
[11]	validation_0-mlogloss:1.73730
[12]	validation_0-mlogloss:1.70069
[13]	validation_0-mlogloss:1.66820
[14]	validation_0-mlogloss:1.63405
[15]	validation_0-mlogloss:1.60492
[16]	validation_0-mlogloss:1.57434
[17]	validation_0-mlogloss:1.54717
[18]	validation_0-mlogloss:1.52182
[19]	validation_0-mlogloss:1.49684
[20]	validation_0-mlogloss:1.47122
[21]	validation_0-mlogloss:1.44978
[22]	validation_0-mlogloss:1.43164
[23]	validation_0-mlogloss:1.41168
[24]	validation_0-mlogloss:1.39337
[25]	validation_0-mlogloss:1.37522
[26]	validation_0-mlogloss:1.35565
[27]	validation_0-mlogloss:1.33657
[28]	validation_0-mlogloss:1.3

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10


9375/9375 [==============================] - 13s 1ms/step - loss: 0.7853 - val_loss: 1.1422
Epoch 2/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.2606 - val_loss: 1.1094
Epoch 3/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.1876 - val_loss: 1.1059
Epoch 4/10
9375/9375 [==============================] - 12s 1ms/step - loss: 0.1566 - val_loss: 1.1714
Epoch 5/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.1365 - val_loss: 1.4676
Epoch 6/10
9375/9375 [==============================] - 12s 1ms/step - loss: 0.1229 - val_loss: 1.3416
Epoch 7/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.1118 - val_loss: 1.2175
Epoch 8/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.1040 - val_loss: 1.1426
Epoch 9/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.0983 - val_loss: 1.2305
Epoch 10/10
9375/9375 [==============================] - 12s 1ms/step - loss: 0.0928

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

  62/1172 [>.............................] - ETA: 0s  

1172/1172 [==============================] - 1s 845us/step
              precision    recall  f1-score   support

           0       0.94      0.71      0.81      1499
           1       0.62      0.89      0.73      1499
           2       0.71      0.82      0.76      1499
           3       0.77      0.66      0.71      1499
           4       0.93      0.93      0.93      1499
           5       0.82      0.97      0.89      1499
           6       0.95      0.91      0.93      1499
           7       0.86      0.81      0.83      1499
           8       0.71      0.85      0.77      1499
           9       0.83      0.92      0.87      1499
          10       0.83      0.93      0.87      1499
          11       0.96      0.92      0.94      1499
          12       0.68      0.86      0.76      1499
          13       0.73      0.94      0.82      1499
          14       0.63      0.73      0.68      1499
          15       0.92      0.26      0.41      1499
          16       0.8

## 0-64

In [26]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [27]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [28]:
model4=XGBClassifier(n_estimators=500)
model4.fit(x,y,early_stopping_rounds=10, eval_set=[(xv, yv)])
y_pred=model4.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.39438
[1]	validation_0-mlogloss:2.13400
[2]	validation_0-mlogloss:1.95841
[3]	validation_0-mlogloss:1.81615
[4]	validation_0-mlogloss:1.70165
[5]	validation_0-mlogloss:1.60782
[6]	validation_0-mlogloss:1.52455
[7]	validation_0-mlogloss:1.45587
[8]	validation_0-mlogloss:1.39223
[9]	validation_0-mlogloss:1.33762
[10]	validation_0-mlogloss:1.28236
[11]	validation_0-mlogloss:1.23707
[12]	validation_0-mlogloss:1.19231
[13]	validation_0-mlogloss:1.15230
[14]	validation_0-mlogloss:1.11473
[15]	validation_0-mlogloss:1.08032
[16]	validation_0-mlogloss:1.05018
[17]	validation_0-mlogloss:1.02044
[18]	validation_0-mlogloss:0.99517
[19]	validation_0-mlogloss:0.96680
[20]	validation_0-mlogloss:0.94445
[21]	validation_0-mlogloss:0.92306
[22]	validation_0-mlogloss:0.90142
[23]	validation_0-mlogloss:0.88116
[24]	validation_0-mlogloss:0.86300
[25]	validation_0-mlogloss:0.84671
[26]	validation_0-mlogloss:0.83118
[27]	validation_0-mlogloss:0.81669
[28]	validation_0-mlogloss:0.8

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.4762 - val_loss: 1.1126
Epoch 2/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.1566 - val_loss: 0.9054
Epoch 3/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.1129 - val_loss: 0.9296
Epoch 4/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.0930 - val_loss: 1.0749
Epoch 5/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.0798 - val_loss: 0.9580
Epoch 6/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.0705 - val_loss: 1.0339
Epoch 7/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.0631 - val_loss: 0.8885
Epoch 8/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.0584 - val_loss: 0.9629
Epoch 9/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.0543 - val_loss: 0.8999
Epoch 10/10
9375/9375 [==============================] - 13s 1ms/step - l

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

 123/1172 [==>...........................] - ETA: 0s

1172/1172 [==============================] - 1s 824us/step
              precision    recall  f1-score   support

           0       0.93      0.73      0.81      1499
           1       0.91      0.96      0.93      1499
           2       0.83      0.78      0.80      1499
           3       0.73      0.98      0.84      1499
           4       0.95      0.99      0.97      1499
           5       0.94      0.99      0.96      1499
           6       0.94      0.99      0.96      1499
           7       0.97      0.77      0.86      1499
           8       0.77      0.88      0.82      1499
           9       0.87      0.85      0.86      1499
          10       0.88      0.69      0.78      1499
          11       0.93      0.90      0.91      1499
          12       0.74      0.92      0.82      1499
          13       0.80      0.97      0.87      1499
          14       0.60      0.88      0.72      1499
          15       0.92      0.39      0.55      1499
          16       0.8

In [15]:
from xgboost import XGBClassifier

model4=XGBClassifier()
model4.fit(x,y)
y_pred=model4.predict(xt)
print(classification_report(yt,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.85      0.79      1499
           1       0.91      0.96      0.93      1499
           2       0.81      0.72      0.76      1499
           3       0.98      0.93      0.95      1499
           4       0.97      0.98      0.98      1499
           5       0.98      0.98      0.98      1499
           6       0.96      0.97      0.96      1499
           7       0.92      0.87      0.90      1499
           8       0.65      0.94      0.77      1499
           9       0.98      0.82      0.89      1499
          10       0.95      0.93      0.94      1499
          11       0.95      0.96      0.96      1499
          12       0.78      0.90      0.84      1499
          13       0.94      0.93      0.93      1499
          14       0.75      0.88      0.81      1499
          15       0.93      0.73      0.81      1499
          16       0.92      0.44      0.60      1499
          17       0.94    

In [23]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators = 500, learning_rate = 0.5,verbosity = 1, random_state = 55)
xgb_model.fit(x,y, eval_set = [(xv,yv)], early_stopping_rounds = 10)
y_pred=xgb_model.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.18258
[1]	validation_0-mlogloss:1.86592
[2]	validation_0-mlogloss:1.68784
[3]	validation_0-mlogloss:1.55050
[4]	validation_0-mlogloss:1.44391
[5]	validation_0-mlogloss:1.34707
[6]	validation_0-mlogloss:1.27419
[7]	validation_0-mlogloss:1.20788
[8]	validation_0-mlogloss:1.14848
[9]	validation_0-mlogloss:1.10192
[10]	validation_0-mlogloss:1.05679
[11]	validation_0-mlogloss:1.01284
[12]	validation_0-mlogloss:0.97881
[13]	validation_0-mlogloss:0.94522
[14]	validation_0-mlogloss:0.91180
[15]	validation_0-mlogloss:0.88008
[16]	validation_0-mlogloss:0.85301
[17]	validation_0-mlogloss:0.82647
[18]	validation_0-mlogloss:0.80048
[19]	validation_0-mlogloss:0.77987
[20]	validation_0-mlogloss:0.76109
[21]	validation_0-mlogloss:0.74467
[22]	validation_0-mlogloss:0.72866
[23]	validation_0-mlogloss:0.71225
[24]	validation_0-mlogloss:0.69797
[25]	validation_0-mlogloss:0.68085
[26]	validation_0-mlogloss:0.66956
[27]	validation_0-mlogloss:0.65700
[28]	validation_0-mlogloss:0.6